In [2]:
#Test with real_video
import matplotlib.pyplot as plt
import os
import random
import time

import tensorflow as tf
import cv2
import numpy as np
import imutils

from segmentation_models.metrics import iou_score
from segmentation_models import Unet
import segmentation_models as sm


sm.set_framework("tf.keras")
sm.framework()

#Initialize parameters
BACKBONE = "resnet34"
preprocess_input = sm.get_preprocessing(BACKBONE)
batch_size = 16
w, h = 384, 384

#initialize model
opt=tf.keras.optimizers.Adam(0.0001)
model= Unet(BACKBONE,encoder_weights = "imagenet",classes = 3,activation="softmax",input_shape = (w, h,3),encoder_freeze = True)
loss1 = sm.losses.categorical_focal_dice_loss

model.compile(optimizer=opt,loss=loss1,metrics=[iou_score])

model.load_weights("best3.hdf5")


Segmentation Models: using `keras` framework.


In [9]:
model.load_weights("best3.hdf5")

ids = range(len(image_val))
index = random.sample(ids, 10)
w, h = 384, 384

import matplotlib.pyplot as plt

for id in index:

    # Input image
    image = cv2.imread(image_val[id])
    image = cv2.resize(image, (w, h))
    # Predict segmentation map with trained model

    mask_predict = model.predict(image[np.newaxis, :, :, :])

    # Read real mask
    image_mask = cv2.imread(mask_val[id], cv2.IMREAD_UNCHANGED)
    image_mask = cv2.resize(image_mask, (w, h))

    plt.figure(figsize=(10, 6))
    plt.subplot(131)
    plt.title("Image")
    plt.imshow(image)
    plt.subplot(132)
    plt.title("Real Polyp")
    plt.imshow(image_mask)
    plt.subplot(133)
    plt.title("Predicted Polyp")
    z = mask_predict[0]#[:, :, 1:2]
    plt.imshow(z)
    plt.show()

NameError: name 'image_val' is not defined

In [10]:

#id = 'http://192.168.1.2:4747/video'
id = 'Polyp_1.mp4'
cap = cv2.VideoCapture(id)
cap1 = cv2.VideoCapture('culture.mp4')
rotate = 0
writer= cv2.VideoWriter('Polyp_1_detected.avi', cv2.VideoWriter_fourcc(*'DIVX'), 20, (w,h))

while True:
    start = time.time()
    success, frame = cap.read()
    if success:
        frame = cv2.resize(frame, (w, h))
        # Predict mask with trained model

        mask_predict = model.predict(frame[np.newaxis, :, :, :])
        mask = mask_predict[0]#[:, :, 0]

        img2 = np.zeros_like(frame)
        img2[:, :, 0] = mask[:, :, 0] * 0
        img2[:, :, 1] = mask[:, :, 1] * 255
        img2[:, :, 2] = mask[:, :, 2] * 255

        #Count FPS
        fps = 1 / (time.time() - start) + 5
        print(start)

        #Draw mask
        result = cv2.bitwise_or(frame, img2)

        #Show prediction
        #im = cv2.hconcat([frame, img2, Result])
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(result, "FPS: " + str(int(fps)), (5, 20), font, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.imshow("Raw video", frame)
        writer.write(result)
        
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
writer.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 31ms/step
1678938911.9663162
1/1 [==============================] - 0s 19ms/step
1678938913.6782136
1/1 [==============================] - 0s 17ms/step
1678938913.754574
1/1 [==============================] - 0s 19ms/step
1678938913.8169613
1/1 [==============================] - 0s 17ms/step
1678938913.8769732
1/1 [==============================] - 0s 19ms/step
1678938913.939559
1/1 [==============================] - 0s 20ms/step
1678938914.0017304
1/1 [==============================] - 0s 17ms/step
1678938914.0631344
1/1 [==============================] - 0s 17ms/step
1678938914.12487
1/1 [==============================] - 0s 17ms/step
1678938914.1867046
1/1 [==============================] - 0s 17ms/step
1678938914.2490423
1/1 [==============================] - 0s 19ms/step
1678938914.311199
1/1 [==============================] - 0s 17ms/step
1678938914.372611
1/1 [==============================] - 0s 17ms/step
1678938914.4341352
1/1 [=======